In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ortools.linear_solver import pywraplp



# Data
# Transportation cost (per unit) matrix  
costs = [
    [51, 62, 35, 45, 56],
    [59, 68, 50, 39, 46],
    [49, 56, 53, 51, 37],
]

num_suppliers = len(costs) #Three in this example
num_markets = len(costs[0]) #Five in this example

Supply = [15, 20, 15] # Supply capacity for supplier A, B and C
Demand = [11, 12, 9, 10, 8] #Demand for market 1 to 5

# Solver
# Create the LP solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')

# Variables
# x[i, j] is an array (continuous) decision variables, which will be
# the number of items supplied from supplier i to market j.​
x = {}
for i in range(num_suppliers):
    for j in range(num_markets):
        x[i, j] =  solver.NumVar(0, solver.infinity(), f"X[{i+1},{j+1}]")
        
       
       
print('Number of variables =', solver.NumVariables()) 
 #Total 15 decision variables



# Constraints
# Each supplier cannot supply will supply exactly equal to it's available capacity
for i in range(num_suppliers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_markets)]) == Supply[i])

# Each market will absorb items exactly equal to its the demand
for j in range(num_markets):
    solver.Add(solver.Sum([x[i, j] for i in range(num_suppliers)]) == Demand[j])

# Objective function
objective_terms = []
for i in range(num_suppliers):
    for j in range(num_markets):
        objective_terms.append(costs[i][j] * x[i, j])

# Call the solver        
solver.Minimize(solver.Sum(objective_terms))



# Solve and print output
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  print('optimal found')
  print('Total cost = ', solver.Objective().Value(), '\n')#printing total cost
  for i in range(num_suppliers):
      for j in range(num_markets):
          print(x[i, j], ' = ', x[i, j].solution_value())#printing each decision variable

Number of variables = 15
optimal found
Total cost =  2316.0 

X[1,1]  =  6.0
X[1,2]  =  0.0
X[1,3]  =  9.0
X[1,4]  =  0.0
X[1,5]  =  0.0
X[2,1]  =  2.0
X[2,2]  =  0.0
X[2,3]  =  0.0
X[2,4]  =  10.0
X[2,5]  =  8.0
X[3,1]  =  3.0
X[3,2]  =  12.0
X[3,3]  =  0.0
X[3,4]  =  0.0
X[3,5]  =  0.0


In [ ]:
#To get the output in the matrix form
print()
print(' ' * 25, '─' * 25)
print(' ' * 25,'Output in the matrix form')    
print(' ' * 25, '─' * 25)
print()
print(' '*12, "Market1", ' '*3,"Market2", ' '*3,"Market3", 
      ' '*3,"Market4", ' '*3,"Market5", ' '*3, "Total Supply")
print('─' * 90)
for i in range(num_suppliers):
      print(f"Supplier {i+1}|", ' '*3,end='')
      for j in range(num_markets):
          print(x[i, j].solution_value(), ' '*8,end='')
      print(Supply[i],end='')    
      print()

print('─' * 90)
print("Total Demand: ", [Demand[i] for i in range(num_markets)])         


                          ─────────────────────────
                          Output in the matrix form
                          ─────────────────────────

             Market1     Market2     Market3     Market4     Market5     Total Supply
──────────────────────────────────────────────────────────────────────────────────────────
Supplier 1|    6.0         0.0         9.0         0.0         0.0         15
Supplier 2|    2.0         0.0         0.0         10.0         8.0         20
Supplier 3|    3.0         12.0         0.0         0.0         0.0         15
──────────────────────────────────────────────────────────────────────────────────────────
Total Demand:  [11, 12, 9, 10, 8]
